In [30]:
import yfinance as yf
import pandas as pd
from common import *

In [31]:
start_date = pd.Timestamp('2025-06-30')
today = pd.Timestamp.today()
eth_ticker = 'ETH-USD'
bitmine_ticker = 'BMNR'

In [32]:
bitmine_holdings = pd.DataFrame({
    'date': [pd.Timestamp('20250630'), pd.Timestamp('20250708'), pd.Timestamp('20250805'), pd.Timestamp('20250811')],
    'eth': [0, 566776, 833137, 1150000]
})

In [33]:
eth_px_data = get_historical_close_px(eth_ticker, start_date=start_date, end_date=today)
assert eth_px_data is not None, "ETH price data is empty"

bitmine_px_data = get_historical_close_px(bitmine_ticker, start_date=start_date, end_date=today)

In [ ]:
# Get share information
bitmine_info = yf.Ticker(bitmine_ticker)
outstanding_shares = bitmine_info.get_shares_full(start=start_date, end=today)

# Convert to Series if it's a DataFrame
if isinstance(outstanding_shares, pd.DataFrame):
    outstanding_shares = outstanding_shares.iloc[:, 0]

# Handle potential None or empty data
if outstanding_shares is None or len(outstanding_shares) == 0:
    # Fallback to basic shares outstanding
    shares = bitmine_info.info.get('sharesOutstanding', 0)
    outstanding_shares = pd.Series([shares], index=[start_date])
    print("Warning: Using current shares outstanding as historical data not available")

# Handle timezone and ensure proper date alignment
outstanding_shares.index = pd.to_datetime(outstanding_shares.index)
if outstanding_shares.index.tz is not None:
    outstanding_shares.index = outstanding_shares.index.tz_localize(None)

# Ensure we have data for every day
outstanding_shares = outstanding_shares.resample('D').ffill()

# Print debug information
print(f"Share data range: {outstanding_shares.index.min()} to {outstanding_shares.index.max()}")
print(f"Number of data points: {len(outstanding_shares)}")
print(f"Sample of share data:\n{outstanding_shares.head()}")

ValueError: cannot reindex on an axis with duplicate labels

In [35]:
bitmine_holdings['eth_px'] = bitmine_holdings['date'].apply(lambda x: get_px_for_date(x, eth_px_data))
bitmine_holdings['stock_px'] = bitmine_holdings['date'].apply(lambda x: get_px_for_date(x, bitmine_px_data))

ValueError: min() iterable argument is empty

In [36]:
# Create daily date range
date_range = pd.date_range(start=start_date, end=today, freq='D')
daily_holdings = pd.DataFrame({'date': date_range})

# Ensure bitmine_holdings dates are timezone-naive
bitmine_holdings['date'] = pd.to_datetime(bitmine_holdings['date']).dt.tz_localize(None)

# Merge with holdings and forward fill ETH
daily_holdings = daily_holdings.merge(bitmine_holdings, on='date', how='left')
daily_holdings['eth'] = daily_holdings['eth'].ffill().round(0)

# Add shares outstanding data
daily_holdings['shares_outstanding'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, outstanding_shares))

# Get daily prices
daily_holdings['eth_px'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, eth_px_data))
daily_holdings['stock_px'] = daily_holdings['date'].apply(lambda x: get_px_for_date(x, bitmine_px_data))

# Calculate metrics
daily_holdings['eth_value_usd_mil'] = ((daily_holdings['eth'] * daily_holdings['eth_px']) / 1_000_000).round(4)
daily_holdings['market_cap_usd_mil'] = ((daily_holdings['stock_px'] * daily_holdings['shares_outstanding']) / 1_000_000).round(4)
daily_holdings['eth_per_share'] = (daily_holdings['eth'] / daily_holdings['shares_outstanding']).round(6)
daily_holdings['eth_value_per_share_usd'] = ((daily_holdings['eth'] * daily_holdings['eth_px']) / daily_holdings['shares_outstanding']).round(4)

# Sort and display
daily_holdings = daily_holdings.sort_values('date')
result = daily_holdings[['date', 'eth', 'eth_px', 'eth_value_usd_mil', 'stock_px', 'shares_outstanding', 
                'market_cap_usd_mil', 'eth_per_share', 'eth_value_per_share_usd']].reset_index(drop=True)
result

ValueError: min() iterable argument is empty